# Reverse Engineering the UR3e IK parameters

Universal Robots posts the [DH parameters](https://www.universal-robots.com/articles/ur/application-installation/dh-parameters-for-calculations-of-kinematics-and-dynamics/) of all of their robots online, the hope is with these parameters for our robot, and the joint position read off the modbus, we might be able to reverse-engineering the inverse kinematics and write our own controller for working with a floating coordinate system.

In [34]:
import numpy as np
import json
import os
import time

np.set_printoptions(formatter={'float': '{: 0.3f}'.format})


In [35]:
DH = np.array([[0, 0, 0.1519,np.pi/2],
                    [0,-0.2434,0.,0.],
                    [0,-0.2132,0.,0.],
                    [0,0.,0.1311,np.pi/2],
                    [0,0.,0.0854,-np.pi/2],
                    [0,0.,0.0921,0.]])

print("Available files:")
files = []
for file in os.listdir():
    if file.endswith(".json"):
        # print("\t" + file)
        files.append(file)

i = 8

path = os.path.abspath("..\\IK_Scans") + "\\" + files[i]

print(path)

with open(path, 'r') as infile:
    json_data = json.load(infile)


start_joints = json_data["Start joints"]
joints_at_points = json_data["joints_at_points"]

print(np.rad2deg(start_joints))
names = ("Floating coordinate system", "Joint angles", "Base position")
for i in range(3):
    print(f"{names[i]}: {joints_at_points[0][i]}")
    
# print(joints_at_points[1])

Available files:
c:\Users\ander\Documents\COBOT-Transducer-Control-Code\IK_Scans\Test_08_11__15_06.json
[[ 166.673]
 [ 307.736]
 [ 94.882]
 [ 228.266]
 [ 273.645]
 [ 107.315]]
Floating coordinate system: [[0, 0, 0], [0, 0, 0]]
Joint angles: [[2.909], [5.371], [1.656], [3.984], [4.776], [1.873]]
Base position: [[[0.4106], [0.0533], [-0.0662]], [[-0.822], [-3.011], [-0.089]]]


In [45]:
#               theta,a,d,alpha     (radians and meters)
DH = np.array([[0, 0, 0.1519,np.pi/2],
                    [0,-0.2434,0.,0.],
                    [0,-0.2132,0.,0.],
                    [0,0.,0.1311,np.pi/2],
                    [0,0.,0.0854,-np.pi/2],
                    [0,0.,0.0921,0.]])

def matrix_to_coordinates(mat):
    '''Accepts as input a 4x4 numpy array representing a transformation matrix
    and returns that transformation in an np array of the form
    [X,Y,Z, beta, alpha, gamma]'''
    X = mat[0,3]
    Y = mat[1,3]
    Z = mat[2,3]

    beta = np.arctan2((mat[0,0]**2 + mat[1,0]**2)**0.5,-mat[2,0])
    cosb=np.cos(beta)
    alpha = np.arctan2(mat[0,0]/cosb, mat[1,0]/cosb)
    gamma = np.arctan2(mat[2,2]/cosb, mat[2,1]/cosb)
    return np.array([X,Y,Z,beta,alpha,gamma])

def joints_forward_kinematics(joints):
    '''Accepts as input a list of 6 joint positions in radians'''
    joints = np.array(joints).flatten()
    DH_params = DH.copy()
    DH_params[:,0] = start_joints

    j_pos = []
    for i in range(6):
        cos_io = np.cos(DH_params[i,0])
        sin_io = np.sin(DH_params[i,0])
        cos_i3 = np.cos(DH_params[i,3])
        sin_i3 = np.sin(DH_params[i,3])

        j_pos.append(np.array([
            [cos_io, -sin_io * cos_i3, sin_io * sin_i3, DH_params[i,1] * cos_io],
            [sin_io, cos_io * cos_i3, -cos_io * sin_i3, DH_params[i,1] * sin_io],
            [0., sin_i3, cos_i3,DH_params[i,2]],
            [0.,0.,0.,1.]
            ]))
    
    # (T1,T2,T3,T4,T5,T6) = tuple(j_pos)
    # t = T1
    t = j_pos[0]
    for i in range(5):
        t = np.matmul(t,j_pos[i+1])
    

    return matrix_to_coordinates(t)


print(joints_forward_kinematics(start_joints))

[ 0.411  0.043  0.107  1.604  2.105  1.627]


In [187]:
a=[1,2,3,4,5,6]
a[-1]

6

In [186]:
Xs,Ys = np.mgrid[-5:5.1:0.5, -5:5.1:0.5]
# print(Xs.shape)
# print(Xs[0],Ys[0])

X,Y,Z = np.mgrid[0:0:1j, -5:5:2, -5:5:2]
# print(X)
# print(Y)
# print(Z)
xyz = np.vstack((X.flatten(),Y.flatten(),Z.flatten())).T
# print(xyz)

xyz2 = np.mgrid[0:0:1j, -5:5:2, -5:5:2].reshape(3,-1).T


xy = np.mgrid[-5:5.1:0.5, -5:5.1:0.5].reshape(2,-1).T

print(xyz.shape)
print(xyz2.shape)
# print(xyz[6])

for i in range(xyz.shape[0]):
    print(f"Compare: {xyz[i] == xyz2[i]}")



(25, 3)
(25, 3)
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]
Compare: [ True  True  True]


In [192]:
a=np.array((1,2,3,4,5))
type(a) is not int

True